# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [148]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [149]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto natales,-51.7236,-72.4875,5.80,75,100,1.25,CL,1733730940
1,1,el calafate,-50.3408,-72.2768,7.69,56,100,2.81,AR,1733730941
2,2,blackmans bay,-43.0167,147.3167,17.63,57,100,1.34,AU,1733730943
3,3,laojunmiao,39.8333,97.7333,-7.53,62,100,2.22,CN,1733730944
4,4,acarau,-2.8856,-40.1200,26.00,82,100,5.26,BR,1733730945
5,5,whitehorse,60.7161,-135.0538,-12.57,92,20,1.54,CA,1733730946
6,6,ilulissat,69.2167,-51.1000,-3.99,46,48,9.77,GL,1733730948
7,7,waitangi,-43.9535,-176.5597,16.57,97,100,3.58,NZ,1733730949
8,8,kodiak,57.7900,-152.4072,5.73,100,100,8.75,US,1733730950
9,9,port-aux-francais,-49.3500,70.2167,2.54,73,87,10.38,TF,1733730951


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [150]:
# Configure the map plot
city_data_humidity_plot = city_data_df.hvplot.points(
    'Lng', 
    'Lat', 
    geo=True, 
    color = 'City', #sets different color for each city
    size = 'Humidity',
)

# Display the map
city_data_humidity_plot


:Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [151]:
# Noting my ideal weather conditions: 
# I don't necessarily care much about cloudiness; one of my favorite cities to visit is Portland, Oregon. 
# I dislike conditions that are simultaneously hot and humid, but don't mind high humidity at a comfortable temperature. Low humidity is fine at any temperature. 
# I hate wind. STOP PUSHING ME!



In [152]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_filtered_df = city_data_df[
    ((city_data_df['Max Temp'] > 75) & (city_data_df['Humidity'] < 50) & (city_data_df['Wind Speed'] >= 8)) | 
    (city_data_df['Max Temp'] < 75 & (city_data_df['Wind Speed'] >= 8))
]
# Drop any rows with null values
city_data_filtered_df = city_data_filtered_df.dropna()

# Display sample data

city_data_filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,laojunmiao,39.8333,97.7333,-7.53,62,100,2.22,CN,1733730944
5,5,whitehorse,60.7161,-135.0538,-12.57,92,20,1.54,CA,1733730946
6,6,ilulissat,69.2167,-51.1000,-3.99,46,48,9.77,GL,1733730948
19,19,vilyuchinsk,52.9306,158.4028,-1.85,91,100,3.04,RU,1733730964
26,26,iqaluit,63.7506,-68.5145,-20.73,70,75,9.26,CA,1733730972
...,...,...,...,...,...,...,...,...,...,...
537,537,thunder bay,48.4001,-89.3168,-2.15,96,100,7.57,CA,1733731627
539,539,berdigestyakh,62.0989,126.6992,-21.51,100,100,1.05,RU,1733731630
548,548,kyshtovka,56.5550,76.6271,-16.55,93,42,3.00,RU,1733731643
552,552,boden,65.8252,21.6886,-6.88,90,18,1.54,SE,1733731648


### Step 3: Create a new DataFrame called `hotel_df`.

In [153]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_filtered_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = None

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
3,3,laojunmiao,39.8333,97.7333,-7.53,62,100,2.22,CN,1733730944,None
5,5,whitehorse,60.7161,-135.0538,-12.57,92,20,1.54,CA,1733730946,None
6,6,ilulissat,69.2167,-51.1000,-3.99,46,48,9.77,GL,1733730948,None
19,19,vilyuchinsk,52.9306,158.4028,-1.85,91,100,3.04,RU,1733730964,None
26,26,iqaluit,63.7506,-68.5145,-20.73,70,75,9.26,CA,1733730972,None
...,...,...,...,...,...,...,...,...,...,...,...
537,537,thunder bay,48.4001,-89.3168,-2.15,96,100,7.57,CA,1733731627,None
539,539,berdigestyakh,62.0989,126.6992,-21.51,100,100,1.05,RU,1733731630,None
548,548,kyshtovka,56.5550,76.6271,-16.55,93,42,3.00,RU,1733731643,None
552,552,boden,65.8252,21.6886,-6.88,90,18,1.54,SE,1733731648,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [154]:
# Set parameters to search for a hotel
category = 'accommodation.hotel'
radius = 10000
limit = 1
params = {"categories": category, "limit": limit, "apiKey": geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    geoapify_response = requests.get(base_url, params)

    # Convert the API response to JSON format
    name_address = geoapify_response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")



Starting hotel search
laojunmiao - nearest hotel: No hotel found
whitehorse - nearest hotel: Town & Mountain Hotel
ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
vilyuchinsk - nearest hotel: Победа
iqaluit - nearest hotel: Frobisher Inn
kuusamo - nearest hotel: Holiday Club Kuusamon Tropiikki
xinmin - nearest hotel: No hotel found
klyuchi - nearest hotel: No hotel found
pangody - nearest hotel: Газпром энерго
iskateley - nearest hotel: Авантаж
kargasok - nearest hotel: Титан
dongfeng - nearest hotel: ****大酒店
yellowknife - nearest hotel: No hotel found
talnakh - nearest hotel: Талнах
vikhorevka - nearest hotel: Сосновые родники
nar'yan-mar - nearest hotel: Гостиница "Агат"
corner brook - nearest hotel: No hotel found
novyy urengoy - nearest hotel: ГазпромТрансгазЮгорск
burang - nearest hotel: No hotel found
labytnangi - nearest hotel: Семь лиственниц
tiksi - nearest hotel: Арктика
kirensk - nearest hotel: Ивушка
olonkinbyen - nearest hotel: No hotel found
slave lake - near

In [155]:
# Display sample data
hotel_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
3,3,laojunmiao,39.8333,97.7333,-7.53,62,100,2.22,CN,1733730944,No hotel found
5,5,whitehorse,60.7161,-135.0538,-12.57,92,20,1.54,CA,1733730946,Town & Mountain Hotel
6,6,ilulissat,69.2167,-51.1000,-3.99,46,48,9.77,GL,1733730948,Best Western Plus Hotel Ilulissat
19,19,vilyuchinsk,52.9306,158.4028,-1.85,91,100,3.04,RU,1733730964,Победа
26,26,iqaluit,63.7506,-68.5145,-20.73,70,75,9.26,CA,1733730972,Frobisher Inn
28,28,kuusamo,65.9667,29.1833,-6.98,93,0,4.12,FI,1733730975,Holiday Club Kuusamon Tropiikki
32,32,xinmin,41.9908,122.8253,-0.68,36,6,2.83,CN,1733730979,No hotel found
34,34,klyuchi,52.2667,79.1667,-17.30,94,91,2.16,RU,1733730982,No hotel found
41,41,pangody,65.8500,74.4859,-12.80,90,99,7.71,RU,1733730990,Газпром энерго
42,42,iskateley,67.6803,53.1512,-5.42,95,100,6.78,RU,1733730992,Авантаж


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [156]:
# Configure the map plot
hotels_plot = hotel_df.hvplot.points(
    'Lng', 
    'Lat', 
    geo=True, 
    color = 'City', #sets different color for each city
    size = 'Humidity',
    hover_cols=['City', 'Country', 'Hotel Name']
)

# Display the map
hotels_plot

:Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)